In [1]:
import dev.johnoreilly.common.data.remote.FantasyPremierLeagueApi
import dev.johnoreilly.common.di.createHttpClient
import dev.johnoreilly.common.model.Player
import io.ktor.client.engine.java.*
import kotlinx.coroutines.runBlocking
import kotlinx.serialization.json.Json
%use kandy
%use dataframe


val json = Json { isLenient = true; ignoreUnknownKeys = true }
val engine = Java.create()
val httpClient = createHttpClient(engine, json, true)
val api = FantasyPremierLeagueApi(httpClient)
 

In [2]:
val staticInfo = runBlocking { api.fetchBootstrapStaticInfo() }
val eventStatus = runBlocking { api.fetchEventStatus() }


In [3]:
val players = staticInfo.elements.sortedByDescending { it.total_points }.take(100)
val player = players[0]

In [4]:
players.toDataFrame()["id", "web_name", "total_points", "selected_by_percent"]

id,web_name,total_points,selected_by_percent
328,M.Salah,271,71.9
182,Palmer,175,61.7
447,Wood,163,44.8
351,Haaland,154,27.4
401,Isak,154,57.9
99,Mbeumo,152,36.1
541,Cunha,132,9.2
58,Watkins,131,20.4
71,Kluivert,128,27.4
311,Alexander-Arnold,121,32.7


In [5]:
val upcomingFixtures = runBlocking { api.fetchUpcomingFixtures() }
upcomingFixtures.toDataFrame()

id,event,kickoff_time,team_h,team_a,team_h_score,team_a_score,team_h_difficulty,team_a_difficulty
257,26,2025-02-21T20:00:00Z,11,4,null,null,2,2
254,26,2025-02-22T12:30:00Z,8,14,null,null,3,3
251,26,2025-02-22T15:00:00Z,1,19,null,null,2,4
253,26,2025-02-22T15:00:00Z,3,20,null,null,2,3
255,26,2025-02-22T15:00:00Z,9,7,null,null,3,3
256,26,2025-02-22T15:00:00Z,10,18,null,null,3,2
260,26,2025-02-22T15:00:00Z,17,5,null,null,3,2
252,26,2025-02-22T17:30:00Z,2,6,null,null,4,3
259,26,2025-02-23T14:00:00Z,15,16,null,null,3,4
258,26,2025-02-23T16:30:00Z,13,12,null,null,5,4


In [6]:
val playerData = runBlocking { api.fetchPlayerData(player.id) }
val playerHistory = playerData.history
val playerHistoryDataFrame = playerHistory.toDataFrame()

In [7]:

playerHistoryDataFrame.plot { 
    layout.title = player.web_name
    bars {
        x(round) { axis.name = "Fixture" }
        y(total_points) { axis.name = "Points" }
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="PWPFKr"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"M.Salah"
},
"mapping":{
},
"data":{
"round":[1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,24.0,25.0,25.0],
"total_points":[14.0,10.0,17.0,2.0,6.0,10.0,3.0,12.0,10.0,9.0,14.0,13.0,13.0,18.0,5.0,21.0,9.0,16.0,7.0,2.0,3.0,8.0,16.0,13.0,8.0,12.0]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"Fixture",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"Points",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"round",
"y":"total_points"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("PWPFKr");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 15 
 
 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 
 
 25 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 15 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 
 
 M.Salah 
 
 
 
 
 Points 
 
 
 
 
 Fixture

In [8]:
val fixtures =  playerHistory.map { it.round }
val totalPoints = playerHistory.map { it.total_points }
val goalsScored = playerHistory.map { it.goals_scored }
val assists = playerHistory.map { it.assists }
val dataset = dataFrameOf("fixture" to fixtures, "totalPoints" to totalPoints,
    "goalsScored" to goalsScored, "assists" to assists
).gather("totalPoints", "goalsScored", "assists").into("stat", "total")

dataset.groupBy("stat").plot {
    layout.title = player.web_name
    layout.size = 800 to 300
    bars {
        x("fixture")
        y("total")
        fillColor("stat") 
        position = Position.stack()
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="tF1CHo"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"M.Salah"
},
"mapping":{
},
"data":{
"fixture":[1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,24.0,25.0,25.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,24.0,25.0,25.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,24.0,25.0,25.0],
"stat":["totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","assists","assists","assists","assists","assists","assists","assists","assists","assists","assists","assists","assists","assists","assists","assists","assists","assists","assists","assists","assists","assists","assists","assists","assists","assists","assists"],
"total":[14.0,10.0,17.0,2.0,6.0,10.0,3.0,12.0,10.0,9.0,14.0,13.0,13.0,18.0,5.0,21.0,9.0,16.0,7.0,2.0,3.0,8.0,16.0,13.0,8.0,12.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,0.0,2.0,1.0,1.0,1.0,0.0,0.0,1.0,2.0,1.0,1.0,1.0,1.0,0.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0],
"&merged_groups":["totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","totalPoints","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","goalsScored","assists","assists","assists","assists","assists","assists","assists","assists","assists","assists","assists","assists","assists","assists","assists","assists","assists","assists","assists","assists","assists","assists","assists","assists","assists","assists"]
},
"ggsize":{
"width":800.0,
"height":300.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true
}],
"layers":[{
"mapping":{
"x":"fixture",
"y":"total",
"fill":"stat",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"position":"stack",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("tF1CHo");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 


In [9]:
staticInfo.events.toDataFrame()

average_entry_score,chip_plays,data_checked,deadline_time,deadline_time_epoch,deadline_time_game_offset,finished,highest_score,highest_scoring_entry,id,is_current,is_next,is_previous,most_captained,most_selected,most_transferred_in,most_vice_captained,name,top_element,top_element_info,transfers_made
57,"[ChipPlayDto(chip_name=bboost, num_pl...",true,2024-08-16T17:30:00Z,1723829400,0,true,127,3546234,1,false,false,false,351,401,27,351,Gameweek 1,328,"TopElementInfoDto(id=328, points=14)",0
69,"[ChipPlayDto(chip_name=bboost, num_pl...",true,2024-08-24T10:00:00Z,1724493600,0,true,154,9442126,2,false,false,false,351,401,594,401,Gameweek 2,177,"TopElementInfoDto(id=177, points=20)",13364453
64,"[ChipPlayDto(chip_name=bboost, num_pl...",true,2024-08-31T10:00:00Z,1725098400,0,true,130,430195,3,false,false,false,351,351,177,328,Gameweek 3,328,"TopElementInfoDto(id=328, points=17)",23723836
51,"[ChipPlayDto(chip_name=bboost, num_pl...",true,2024-09-14T10:00:00Z,1726308000,0,true,120,3560750,4,false,false,false,351,351,129,328,Gameweek 4,185,"TopElementInfoDto(id=185, points=15)",25930392
58,"[ChipPlayDto(chip_name=bboost, num_pl...",true,2024-09-21T10:00:00Z,1726912800,0,true,131,1773336,5,false,false,false,351,351,58,351,Gameweek 5,180,"TopElementInfoDto(id=180, points=16)",13036058
50,"[ChipPlayDto(chip_name=bboost, num_pl...",true,2024-09-28T10:00:00Z,1727517600,0,true,142,7479996,6,false,false,false,351,351,327,328,Gameweek 6,182,"TopElementInfoDto(id=182, points=25)",21036913
46,"[ChipPlayDto(chip_name=bboost, num_pl...",true,2024-10-05T10:00:00Z,1728122400,0,true,130,759474,7,false,false,false,351,351,182,328,Gameweek 7,512,"TopElementInfoDto(id=512, points=18)",14594461
36,"[ChipPlayDto(chip_name=bboost, num_pl...",true,2024-10-19T10:00:00Z,1729332000,0,true,98,5482232,8,false,false,false,351,351,491,328,Gameweek 8,228,"TopElementInfoDto(id=228, points=14)",11911632
54,"[ChipPlayDto(chip_name=bboost, num_pl...",true,2024-10-25T17:30:00Z,1729877400,0,true,117,2039494,9,false,false,false,351,351,99,182,Gameweek 9,99,"TopElementInfoDto(id=99, points=15)",10882045
39,"[ChipPlayDto(chip_name=bboost, num_pl...",true,2024-11-02T11:00:00Z,1730545200,0,true,111,1689659,10,false,false,false,351,351,99,182,Gameweek 10,82,"TopElementInfoDto(id=82, points=16)",11763382


In [10]:
staticInfo.teams.toDataFrame()

code,draw,form,id,loss,name,played,points,position,pulse_id,short_name,strength,strength_attack_away,strength_attack_home,strength_defence_away,strength_defence_home,strength_overall_away,strength_overall_home,unavailable,win
3,0,null,1,0,Arsenal,0,0,2,1,ARS,4,1370,1220,1300,1250,1335,1235,false,0
7,0,null,2,0,Aston Villa,0,0,9,2,AVL,3,1160,1080,1270,1140,1215,1110,false,0
91,0,null,3,0,Bournemouth,0,0,5,127,BOU,4,1170,1130,1260,1260,1215,1195,false,0
94,0,null,4,0,Brentford,0,0,11,130,BRE,3,1080,1080,1310,1120,1195,1100,false,0
36,0,null,5,0,Brighton,0,0,10,131,BHA,3,1140,1120,1160,1160,1150,1140,false,0
8,0,null,6,0,Chelsea,0,0,6,4,CHE,4,1150,1150,1320,1310,1235,1230,false,0
31,0,null,7,0,Crystal Palace,0,0,13,6,CRY,3,1140,1140,1120,1120,1130,1130,false,0
11,0,null,8,0,Everton,0,0,14,7,EVE,3,1140,1130,1100,1070,1125,1095,false,0
54,0,null,9,0,Fulham,0,0,8,34,FUL,3,1150,1150,1150,1150,1150,1150,false,0
40,0,null,10,0,Ipswich,0,0,18,8,IPS,3,1070,1070,1080,1080,1075,1075,false,0
